该文件主要做把数据导出为josn 或者 excel格式的表，以供求解器容易读取数据

In [91]:
import pandas as pd

### Car rental

In [3]:
# 定义集合
depots = ['Glasgow','Manchester','Birmingham','Plymouth']
days = [0,1,2,3,4,5] 
rentDays = [1,2,3]


In [4]:
demand_df = pd.DataFrame(columns=depots)
demands = [[100,250,95,160],
           [150,143,195,99],
           [135,80,242,55],
           [83,225,111,96],
           [120,210,70,115],
           [230,98,124,80]]
for day in days:
    demand_df.loc[day] = demands[day]
demand_df.to_excel("../data/car_rental/demands.xlsx")
demand_df

,Glasgow,Manchester,Birmingham,Plymouth
0,100,250,95,160
1,150,143,195,99
2,135,80,242,55
3,83,225,111,96
4,120,210,70,115
5,230,98,124,80


In [98]:
dddd = pd.read_excel("../data/car_rental/demands.xlsx",index_col=0)
dddd

,Glasgow,Manchester,Birmingham,Plymouth
0,100,250,95,160
1,150,143,195,99
2,135,80,242,55
3,83,225,111,96
4,120,210,70,115
5,230,98,124,80


In [5]:
#repairCap
capacities = [0,12,20,0]
capacity_df = pd.DataFrame(columns=["capacity"])
i = 0
for depot in depots:
    capacity_df.loc[depot,"capacity"] = capacities[i]
    i += 1
capacity_df.to_excel("../data/car_rental/repair_capacity.xlsx")
capacity_df

,capacity
Glasgow,0
Manchester,12
Birmingham,20
Plymouth,0


In [99]:
dddd = pd.read_excel("../data/car_rental/repair_capacity.xlsx",index_col=0)
dddd

,capacity
Glasgow,0
Manchester,12
Birmingham,20
Plymouth,0


['Manchester', 'Birmingham']

In [6]:
rent_to_return = [[0.6,0.2,0.1,0.1],
                 [0.15,0.55,0.25,0.05],
                 [0.15,0.2,0.54,0.11],
                 [0.08,0.12,0.27,0.53]]
i = 0
rent_to_return_df = pd.DataFrame(columns=depots)
for depot in depots:
    rent_to_return_df.loc[depot] = rent_to_return[i]
    i += 1    
rent_to_return_df.to_excel("../data/car_rental/rent_to_return.xlsx")
rent_to_return_df

,Glasgow,Manchester,Birmingham,Plymouth
Glasgow,0.60,0.20,0.10,0.10
Manchester,0.15,0.55,0.25,0.05
Birmingham,0.15,0.20,0.54,0.11
Plymouth,0.08,0.12,0.27,0.53


In [7]:
transfers = [[0,20,30,50],
            [20,0,15,35],
            [30,15,0,25],
            [50,35,25,0]]
i = 0
transfers_df = pd.DataFrame(columns=depots)
for depot in depots:
    transfers_df.loc[depot] = transfers[i]
    i += 1    
transfers_df.to_excel("../data/car_rental/transfers.xlsx")
transfers_df

,Glasgow,Manchester,Birmingham,Plymouth
Glasgow,0,20,30,50
Manchester,20,0,15,35
Birmingham,30,15,0,25
Plymouth,50,35,25,0


In [8]:
#### 其他参数
other_cost = [[0.55,20,50,70],
             [0.20,25,70,100],
             [0.25,30,120,150]]
other_cost_df = pd.DataFrame(columns=["Q","CS","RCA","RCB"])
for k in rentDays:
    other_cost_df.loc[k] = other_cost[k-1]
other_cost_df.to_excel("../data/car_rental/other_costs.xlsx")
    
other_cost_df

,Q,CS,RCA,RCB
1,0.55,20.0,50.0,70.0
2,0.20,25.0,70.0,100.0
3,0.25,30.0,120.0,150.0


In [9]:

PD = 0.1
CO = 15
DF = 10

In [10]:
import pyscipopt as scip

In [70]:
model = scip.Model()

In [71]:
# 变量定义
n = model.addVar(name="num_cars")
nu = dict()
nd = dict()
tr = dict()
eu = dict()
ed = dict()
rp = dict()
for depot in depots:
    for day in days:
        nu[(depot,day)] = model.addVar(name="num_ud_cars(%s,%s)"%(depot,day))
        nd[(depot,day)] = model.addVar(name="num_d_cars(%s,%s)"%(depot,day))
        tr[(depot,day)] = model.addVar(name="num_h_cars(%s,%s)"%(depot,day),lb=0,ub=demand_df.loc[day,depot])
        eu[(depot,day)] = model.addVar(name="end_inv_ud(%s,%s)"%(depot,day))
        ed[(depot,day)] = model.addVar(name="end_inv_d(%s,%s)"%(depot,day))
        rp[(depot,day)] = model.addVar(name="repair_num_cars(%s,%s)"%(depot,day),lb=0,ub=capacity_df.loc[depot,"capacity"])
        
tu = dict()
td = dict()
for depot_i in depots:
    for depot_j in depots:
        for day in days:
            tu[(depot_i,depot_j,day)] = model.addVar(name="trans_ud(%s,%s,%s)"%(depot_i,depot_j,day))
            td[(depot_i,depot_j,day)] = model.addVar(name="trans_d(%s,%s,%s)"%(depot_i,depot_j,day))


In [72]:
# 约束
## 1、2
for depot in depots:
    for day in days:
        pre_day = (day-1)%6
        model.addCons(scip.quicksum((1-PD)*rent_to_return_df.loc[other_depot,depot] * other_cost_df.loc[k,"Q"] * tr[(other_depot,(day-k)%6)] 
                                   for other_depot in depots for k in rentDays) 
                      + scip.quicksum(tu[(other_depot,depot,pre_day)] for other_depot in depots)
                     + rp[(depot,pre_day)] + eu[(depot,pre_day)] == nu[(depot,day)])

        model.addCons(scip.quicksum(PD*rent_to_return_df.loc[other_depot,depot]* other_cost_df.loc[k,"Q"] * tr[(other_depot,(day-k)%6)] 
                                   for other_depot in depots for k in rentDays) 
                     + scip.quicksum(td[(other_depot,depot,pre_day)] for other_depot in depots) + ed[(depot,pre_day)] == nd[(depot,day)])

#         model.addCons(tr[(depot,day)] + scip.quicksum(tu[(depot,other_depot,day)] for other_depot in depots) + rp[(depot,pre_day)] 
#                       + eu[(depot,day)] == nu[(depot,day)])
        model.addCons(tr[(depot,day)] + scip.quicksum(tu[(depot,other_depot,day)] for other_depot in depots)
                      + eu[(depot,day)] == nu[(depot,day)])        
        model.addCons(scip.quicksum(td[(depot,other_depot,day)] for other_depot in depots) + rp[(depot,pre_day)] 
                      + ed[(depot,day)] == nd[(depot,day)])


model.addCons(scip.quicksum(0.25*tr[(depot,0)] + 0.45*tr[(depot,1)] + nu[(depot,2)] + nd[(depot,2)] for depot in depots) == n)

c97

In [73]:
obj_1 = scip.quicksum(rent_to_return_df.loc[depot,depot]*other_cost_df.loc[k,"Q"]*
                      (other_cost_df.loc[k,"RCA"] -other_cost_df.loc[k,"CS"] + DF)*tr[(depot,day)] for depot in depots for k in rentDays
                     for day in days)

obj_2 = scip.quicksum(rent_to_return_df.loc[depot_i,depot_j]*other_cost_df.loc[k,"Q"]*
                     (other_cost_df.loc[k,"RCB"] -other_cost_df.loc[k,"CS"] + DF)*tr[(depot,day)] 
                      for depot_i in depots for depot_j in depots for k in rentDays for day in days)

obj_3 = scip.quicksum(transfers_df.loc[depot_i,depot_j]*(tu[(depot_i,depot_j,day)] +td[(depot_i,depot_j,day)]) 
                      for depot_i in depots for depot_j in depots for day in days)
obj = obj_1 + obj_2 - obj_3 - CO*n
model.setObjective(obj,"maximize")

In [74]:
model.optimize()

### 结果分析

In [75]:
# Output report

# Total number of cars owned
print(f"The optimal number of cars to be owned is: {round(model.getVal(n))}.")

# Optimal profit
print(f"The optimal profit is: {'${:,.2f}'.format(round(model.getObjVal(),2))}.")

The optimal number of cars to be owned is: 636.
The optimal profit is: $243,912.29.


In [63]:
# Create a list to translate the number label of each day to the actual name of the day
dayname = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']

# Number of undamaged cars in depot at the beginning of each day.
print("\n\n_________________________________________________________________________________")
print(f"Estimated number of undamaged cars in depot at the beginning of each day: ")
print("_________________________________________________________________________________")

undamaged_cars = pd.DataFrame(columns=['Day','Glasgow','Manchester','Birmingham','Plymouth'])
for t in days:
    undamaged_cars = undamaged_cars.append({"Day": dayname[t], "Glasgow": round(model.getVal(nu['Glasgow',t])),
                                            "Manchester":  round(model.getVal(nu['Manchester',t])),
                                            'Birmingham':  round(model.getVal(nu['Birmingham',t])),
                                            'Plymouth': round(model.getVal(nu['Plymouth',t]))   }, ignore_index=True) 
undamaged_cars.index=[''] * len(undamaged_cars)
undamaged_cars



_________________________________________________________________________________
Estimated number of undamaged cars in depot at the beginning of each day: 
_________________________________________________________________________________


,Day,Glasgow,Manchester,Birmingham,Plymouth
,Monday,58,87,87,160
,Tuesday,62,92,106,99
,Wednesday,63,93,109,68
,Thursday,63,103,108,96
,Friday,63,93,104,115
,Saturday,63,92,104,96


In [64]:
# Number of Damaged cars in depot at the beginning of each day.
print("_________________________________________________________________________________")
print(f"Estimated number of damaged cars in depot at the beginning of each day: ")
print("_________________________________________________________________________________")

damaged_cars = pd.DataFrame(columns=['Day','Glasgow','Manchester','Birmingham','Plymouth'])
for t in days:
    damaged_cars = damaged_cars.append({"Day": dayname[t], "Glasgow": round(model.getVal(nd['Glasgow',t])),
                                        "Manchester":  round(model.getVal(nd['Manchester',t])),
                                        'Birmingham':  round(model.getVal(nd['Birmingham',t])),
                                        'Plymouth': round(model.getVal(nd['Plymouth',t]))   }, ignore_index=True) 
damaged_cars.index=[''] * len(damaged_cars)
damaged_cars

_________________________________________________________________________________
Estimated number of damaged cars in depot at the beginning of each day: 
_________________________________________________________________________________


,Day,Glasgow,Manchester,Birmingham,Plymouth
,Monday,6,12,20,7
,Tuesday,7,12,20,9
,Wednesday,7,14,20,8
,Thursday,7,12,23,7
,Friday,7,15,20,7
,Saturday,7,12,20,10


In [68]:
# Undamaged car rented out from each depot and day.
print("_________________________________________________________________________________")
print(f"Estimated number of undamaged cars rented out from each depot and day: ")
print("_________________________________________________________________________________")

rentedOut = {}

for d in depots:
    for t in days:
        count = 0
        for d2 in depots:
            for r in rentDays:
                #print(f"Depot {d}, day {t}: cars rented out {tr[d,t].x}")
                count += (1-PD)*rent_to_return_df.loc[d,d2]*other_cost_df.loc[r,"Q"]*model.getVal(tr[d,t])
        rentedOut[d,t] = round(count)
    

#print(rentedOut)

rentout_cars = pd.DataFrame(columns=['Day','Glasgow','Manchester','Birmingham','Plymouth'])
for t in days:
    rentout_cars = rentout_cars.append({"Day": dayname[t], "Glasgow": round(rentedOut['Glasgow',t]), "Manchester":  round(rentedOut['Manchester',t]), 'Birmingham':  round(rentedOut['Birmingham',t]),'Plymouth': round(rentedOut['Plymouth',t])   }, ignore_index=True) 
rentout_cars.index=[''] * len(rentout_cars)
rentout_cars

_________________________________________________________________________________
Estimated number of undamaged cars rented out from each depot and day: 
_________________________________________________________________________________


,Day,Glasgow,Manchester,Birmingham,Plymouth
,Monday,52,78,60,144
,Tuesday,56,83,95,89
,Wednesday,57,72,78,50
,Thursday,56,93,47,86
,Friday,57,84,63,104
,Saturday,57,83,16,72


In [82]:
x = dict()

In [83]:
x["a"] = 1

In [84]:
x["b"] = 2

In [86]:
x[("x","y")] = 2

In [87]:
x

{'a': 1, 'b': 2, ('x', 'y'): 2}

In [88]:
x[("x","y")] = 5

In [89]:
x

{'a': 1, 'b': 2, ('x', 'y'): 5}

In [1]:
import pyscipopt as scip

In [2]:
model = scip.Model()

In [3]:
help(model.addConsIndicator)

Help on built-in function addConsIndicator:

addConsIndicator(...) method of pyscipopt.scip.Model instance
    Add an indicator constraint for the linear inequality 'cons'.
    
    The 'binvar' argument models the redundancy of the linear constraint. A solution for which
    'binvar' is 1 must satisfy the constraint.
    
    :param cons: a linear inequality of the form "<="
    :param binvar: binary indicator variable, or None if it should be created (Default value = None)
    :param name: name of the constraint (Default value = "IndicatorCons")
    :param initial: should the LP relaxation of constraint be in the initial LP? (Default value = True)
    :param separate: should the constraint be separated during LP processing? (Default value = True)
    :param enforce: should the constraint be enforced during node processing? (Default value = True)
    :param check: should the constraint be checked for feasibility? (Default value = True)
    :param propagate: should the constraint be pr